In [1]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import numpy as np
x = ['1','2','3','4']
y1 = np.array((32.9, 18.5, 17.5, 18.4))
y2 = np.array([25.7 ,17.3 ,16, 16.1])
y3 = np.array([20.9, 22.9 ,18.0, 18.3])
y4 = np.array([20.9, 18.5, 17.5, 15.8])

fig, ax = plt.subplots()
ax.plot(x, y1, label='With print')
ax.plot(x, y2, label='With Logger')
ax.plot(x, y3, label='With lists')
ax.plot(x, y4, label='With global variables')

ax.legend()
ax.set_xlabel('Execucão')
ax.set_ylabel('Tempo de execucão')
ax.set_title('Impacto de modificações na performance')

plt.show()


,id
0,2673569085
1,2673573278
2,2672190040
3,2635594811
4,2635606760
5,2673419890
6,2648856912
7,2673449882
8,2238085516
9,2672499405


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x = ['1','2','3','4']
y1 = np.array((32.9, 18.5, 17.5, 18.4))
y2 = np.array([25.7 ,17.3 ,16, 16.1])
y3 = np.array([20.9, 22.9 ,18.0, 18.3])
y4 = np.array([20.9, 18.5, 17.5, 15.8])

fig, ax = plt.subplots()
ax.plot(x, y1, label='With print')
ax.plot(x, y2, label='With Logger')
ax.plot(x, y3, label='With lists')
ax.plot(x, y4, label='With global variables')

ax.legend()
ax.set_xlabel('Execucão')
ax.set_ylabel('Tempo de execucão')
ax.set_title('Impacto de modificações na performance')

plt.show()


,id
0,2673569085
1,2673573278
2,2672190040
3,2635594811
4,2635606760
5,2673419890
6,2648856912
7,2673449882
8,2238085516
9,2672499405


In [4]:
df

,id
0,2673569085
1,2673573278
2,2672190040
3,2635594811
4,2635606760
5,2673419890
6,2648856912
7,2673449882
8,2238085516
9,2672499405
